# Ridge, LASSO, and Elastic Net Regression

### Dataset and prep

In [ ]:
from sklearn.datasets import load_boston

boston = load_boston()

In [ ]:
print(boston.feature_names)
print()
print(type(boston.feature_names))
print()
print(len(boston.feature_names))

In [ ]:
print(boston.data)
print()
print(type(boston.data))
print()
print(len(boston.data))

In [ ]:
print(boston.target)
print()
print(type(boston.target))
print()
print(len(boston.target))

### Train/Test Split

In [ ]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target,
                                                    train_size=0.8, test_size=0.2)

### Models

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
import numpy as np

np.random.seed(50)

lin_reg = LinearRegression()

ridge_reg = Ridge(alpha=.1,  # regularization (L2) strength
                  max_iter=1000,
                  tol=0.0001,  # when to stop
                  random_state=40,
                  solver='auto')  # or ‘svd’, ‘cholesky’, ‘lsqr’, ‘sparse_cg’, ‘sag’

lasso_reg = Lasso(alpha=.1,  # multiples L1 penalty, a 0 is linreg
                  max_iter=1000,
                  tol=0.0001,  # when to stop
                  random_state=40,
                  selection='cyclic')  # how to update coefficients, also 'random'

en_reg = ElasticNet(alpha=.1,  # multiplies penalty, 0 is linreg
                    l1_ratio=0.5,  # ratio of L1 in penalty, 0 would be pure L2 penalty, mix of ridge and lasso
                    max_iter=1000,
                    tol=0.0001,  # when to stop
                    random_state=40,
                    selection='cyclic')  # how to update coefficients, also 'random'

lin_model = lin_reg.fit(X_train, y_train)
ridge_model = ridge_reg.fit(X_train, y_train)
lasso_model = lasso_reg.fit(X_train, y_train)
en_model = lasso_reg.fit(X_train, y_train)


print("Linear R2:\n")
print(lin_model.score(X_test, y_test))  # R2
print()
print("Ridge R2:\n")
print(ridge_model.score(X_test, y_test))  # R2
print()
print("Lasso R2:\n")
print(lasso_model.score(X_test, y_test))  # R2
print()
print("Elastic Net R2:\n")
print(en_model.score(X_test, y_test))  # R2

### Compare coefficients

In [ ]:
for model in (lin_model, ridge_model, lasso_model, en_model):
    print(model)
    print()
    print (model.coef_)
    print()

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

y_pred_lin = lin_reg.fit(X_train, y_train).predict(X_test)
y_pred_en = en_reg.fit(X_train, y_train).predict(X_test)
y_pred_lasso = lasso_reg.fit(X_train, y_train).predict(X_test)
y_pred_ridge = ridge_reg.fit(X_train, y_train).predict(X_test)

plt.plot(ridge_model.coef_, color='green', linewidth=2, label='Linear Reg. coefficients')
plt.plot(ridge_model.coef_, color='blue', linewidth=2, label='Ridge coefficients')
plt.plot(lasso_model.coef_, color='gold', linewidth=2, label='Lasso coefficients')
plt.plot(en_model.coef_, color='red', linewidth=2, label='Elastic Net coefficients')

# plt.plot(coef, '--', color='navy', label='original coefficients')
plt.legend(loc='best')
plt.title("Linear Reg. R^2: %f, Ridge Net R^2: %f, Lasso R^2: %f, Elastic Net R^2: %f"
          % (r2_score(y_test, y_pred_lin), r2_score(y_test, y_pred_ridge), r2_score(y_test, y_pred_lasso), r2_score(y_test, y_pred_en)))
plt.show()

### CV Models

In [ ]:
from sklearn.model_selection import cross_val_score
lin_scores = cross_val_score(lin_reg, boston.data, boston.target, cv=5)
ridge_scores = cross_val_score(ridge_reg, boston.data, boston.target, cv=5)
lasso_scores = cross_val_score(lasso_reg, boston.data, boston.target, cv=5)
en_scores = cross_val_score(en_reg, boston.data, boston.target, cv=5)

print("Linear Reg. CV R2:\n")
print(lin_scores, "---Average:", np.mean(lin_scores))  # R2
print()
print("Ridge CV R2:\n")
print(ridge_scores, "---Average:", np.mean(ridge_scores))  # R2
print()
print("Lasso CV R2:\n")
print(lasso_scores, "---Average:", np.mean(lasso_scores))  # R2
print()
print("Elastic Net CV R2:\n")
print(en_scores, "---Average:", np.mean(en_scores))  # R2

In [ ]:
r_s_ridge = []

for i in np.arange(0, 1, .01):

    ridge_reg = Ridge(alpha=i,  # regularization (L2) strength
                      max_iter=1000,
                      tol=0.0001,  # when to stop
                      random_state=40,
                      solver='auto')  # or ‘svd’, ‘cholesky’, ‘lsqr’, ‘sparse_cg’, ‘sag’
    
    ridge_model = ridge_reg.fit(X_train, y_train)
    
    r_s_ridge.append((i,np.mean(cross_val_score(ridge_reg, boston.data, boston.target, cv=5))))
    
print(sorted(r_s_ridge, key=lambda tup: tup[1], reverse=True)[0])

In [ ]:
r_s_lasso = []

for i in np.arange(0, 1, .01):

    lasso_reg = Lasso(alpha=i,  # multiples L1 penalty, a 0 is linreg
                      max_iter=1000,
                      tol=0.0001,  # when to stop
                      random_state=40,
                      selection='cyclic')  # how to update coefficients, also 'random'
    
    lasso_model = lasso_reg.fit(X_train, y_train)
    
    r_s_lasso.append((i,np.mean(cross_val_score(lasso_reg, boston.data, boston.target, cv=5))))
    
print(sorted(r_s_lasso, key=lambda tup: tup[1], reverse=True)[0])

In [ ]:
r_s_en = []

for i in np.arange(0, 1, .01):

    en_reg = ElasticNet(alpha=i,  # multiplies penalty, 0 is linreg
                        l1_ratio=0.5,  # ratio of L1 in penalty, 0 would be pure L2 penalty, mix of ridge and lasso
                        max_iter=1000,
                        tol=0.0001,  # when to stop
                        random_state=40,
                        selection='cyclic')  # how to update coefficients, also 'random'

    en_model = lasso_reg.fit(X_train, y_train)
    r_s_en.append((i,np.mean(cross_val_score(en_reg, boston.data, boston.target, cv=5))))
    
print(sorted(r_s_en, key=lambda tup: tup[1], reverse=True)[0])